## LangChain Runnables

Runnable as the basic, chainable building block

-   Prompt Template
-   Chat Model
-   Output Parsers
-   etc.


#### 1. RunnableLambda

The RunnableLambda allows you to wrap any standard Python function (including lambda functions) and integrate it directly into an LCEL chain. It's the bridge between raw Python logic and the LangChain ecosystem.


`Wrapping a Single Function`


In [1]:
from langchain_core.runnables import RunnableLambda


def reverse(s: str) -> str:
    # A simple Python function to reverse a string
    return s[::-1]


# Wrap the function to turn it into a Runnable
runnable = RunnableLambda(func=reverse)

# Invoke the Runnable directly
print(f"Output: {runnable.invoke('Hello')}")

Output: olleH


/Users/yashjain/Developer/youtube/langchain-youtube/.langchain-youtube-env/lib/python3.14/site-packages/langchain_core/_api/deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


`Chaining Multiple RunnableLambda Functions`


In [3]:
from langchain_core.runnables import RunnableLambda


def reverse(s: str) -> str:
    # 1. Reverses the string
    return s[::-1]


def convert_title(s: str) -> str:
    # 2. Converts the string to Title Case
    return s.title()


runnable_1 = RunnableLambda(func=reverse)
runnable_2 = RunnableLambda(func=convert_title)

# Chain them using the pipe operator (|)
# Input 'Hello' -> 'olleH' (by runnable_1) -> 'Olleh' (by runnable_2)
chain = runnable_1 | runnable_2


print(f"Output: {chain.invoke('Hello')}")

olleH
Output: Olleh


---

#### 2. RunnablePassthrough

The RunnablePassthrough is used to pass the entire input through to the next step, often combined with dictionary mapping to create parallel or complex chains without losing the original input data.


In [6]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough


def reverse(s: str) -> str:
    return s[::-1]


def convert_title(_dict: dict[str, str]) -> dict:
    # This function now takes the dictionary output from the previous step
    # and converts it to a title-cased string for display.
    return {"output": _dict["output"].title(), "temp": _dict}


runnable_1 = RunnableLambda(func=reverse)
runnable_2 = RunnableLambda(func=convert_title)

# Chain Breakdown:
# 1. runnable_1 gets "Hello" and outputs "olleH".
# 2. {"output": RunnablePassthrough()} takes "olleH" (the input to this step)
#    and creates a dictionary: {"output": "olleH"}.
# 3. runnable_2 gets the dictionary {"output": "olleH"}.
chain = runnable_1 | {"output": RunnablePassthrough(), "output_2": RunnablePassthrough()} | runnable_2


print(f"Output: {chain.invoke('Hello')}")

Output: {'output': 'Olleh', 'temp': {'output': 'olleH', 'output_2': 'olleH'}}


---

#### 3. RunnableParallel

The RunnableParallel allows you to run multiple Runnables concurrently (at the same time) using the same input. The output is always a dictionary where the keys are the names you give to each branch.


`Simple Parallel Execution`


In [7]:
from langchain_core.runnables import RunnableLambda, RunnableParallel


def add_ten(x: dict) -> dict:
    # Assumes input is a dictionary like {"input": 10}
    return {"added": x["input"] + 10}


# Define the parallel branches using a dictionary
# The input {"input": 10} is sent to both branches simultaneously.
mapper = RunnableParallel(
    {
        "add_step": RunnableLambda(add_ten),
        "add_step_2": RunnableLambda(add_ten),
    },
)


print(f"Output: {mapper.invoke({'input': 10})}")

Output: {'add_step': {'added': 20}, 'add_step_2': {'added': 20}}


`Parallel Processing After a Sequential Step (LCEL Syntax)`


In [8]:
from langchain_core.runnables import RunnableLambda, RunnableParallel


def add_one(x: int) -> int:
    return x + 1


def mul_two(x: int) -> int:
    # Takes the output of add_one (2) and multiplies it
    return x * 2


def mul_three(x: int) -> int:
    # Takes the output of add_one (2) and multiplies it
    return x * 3


runnable_1 = RunnableLambda(add_one)
runnable_2 = RunnableLambda(mul_two)
runnable_3 = RunnableLambda(mul_three)

# Chain Breakdown:
# 1. runnable_1 gets 1, outputs 2.
# 2. The dictionary acts as a RunnableParallel:
#    It takes 2 as input and sends it to runnable_2 and runnable_3 simultaneously.
# Note: RunnableParallel can be created with either dictionary syntax or keyword args.

# Using dictionary syntax in LCEL:
chain_dict = runnable_1 | {"mul_two": runnable_2, "mul_three": runnable_3}
print(f"Output: {chain_dict.invoke(1)}")

Output: {'mul_two': 4, 'mul_three': 6}


In [9]:
# Using dictionary syntax in LCEL:
chain_dict = runnable_1 | RunnableParallel({"mul_two": runnable_2, "mul_three": runnable_3})
print(f"Output: {chain_dict.invoke(1)}")

Output: {'mul_two': 4, 'mul_three': 6}


In [10]:
# Using RunnableParallel with keyword arguments (equivalent to the above):
chain_kwargs = runnable_1 | RunnableParallel(mul_two=runnable_2, mul_three=runnable_3)
print(f"Output: {chain_kwargs.invoke(1)}")

Output: {'mul_two': 4, 'mul_three': 6}


---

#### Some Examples


In [11]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableParallel

runnable = RunnableParallel(
    {
        "joke": PromptTemplate.from_template("Tell me a joke about {topic}"),
        "poem": PromptTemplate.from_template("Tell me a poem about {topic}"),
    },
)

runnable.invoke({"topic": "code"})

{'joke': StringPromptValue(text='Tell me a joke about code'),
 'poem': StringPromptValue(text='Tell me a poem about code')}

In [12]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnableLambda


# This RunnableLambda capitalizes the input topic before it hits the prompts.
def preprocess_input(input_dict: dict[str, str]) -> dict[str, str]:
    """Converts the topic value to uppercase."""
    # The output MUST be a dictionary that matches the input keys required by the next step.
    return {"topic": input_dict["topic"].upper()}


# Define the prompts that will run in parallel using the same input key ('topic').
parallel_prompts = RunnableParallel(
    {
        "synonym": PromptTemplate.from_template("What is a synonym for {topic}?"),
        "antonym": PromptTemplate.from_template("What is an antonym for {topic}?"),
    },
)

# The chain first runs the lambda, then immediately forks the output to the parallel prompts.
chain = RunnableLambda(preprocess_input) | parallel_prompts


# Input: {"topic": "happy"}
# Step 1 (Lambda) Output: {"topic": "HAPPY"}
# Step 2 (Parallel) runs both prompts with "HAPPY"
result = chain.invoke({"topic": "happy"})

print(result)

{'synonym': StringPromptValue(text='What is a synonym for HAPPY?'), 'antonym': StringPromptValue(text='What is an antonym for HAPPY?')}
